# ***Part 1:*** *General code and definitions*

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [81]:
# evaluationg evalf and evalf

def evalf(N,x):
  assert type(N) is int and N >0
  assert type(x) is np.ndarray and len(x) == N
  x = x.reshape((N,1))
  a = np.array([10**-i for i in range(1,N+1)]).reshape((1,N))
  b = np.arange(1,N+1).reshape((N,1))
  f = np.dot(a,(x-b)**2)
  return float(f)

def evalg(N,x):
  assert type(N) is int and N >0
  assert type(x) is np.ndarray and len(x) == N
  c = np.arange(1,N+1)
  d = np.array([10**i for i in range(1,N+1)])
  g = 2*(x - c) / d
  return g



The equation $h(x)$ can be written as follows siince its in quadratic form :

 \begin{array}{l}


 \\ \\  \\

h( X) \ =\ \sum ^{n}_{i=1}\frac{1}{10^{i}}( x_{i} -i)^{2}\\
\\
=\sum ^{n}_{i=1}\frac{1}{10^{i}}\left( x^{2}_{i} +i^{2} -2x_{i} .i\right)\\
=\begin{bmatrix}
x_{1} & \dotsc  & ... & x_{n}
\end{bmatrix}\begin{bmatrix}
\frac{1}{10} &  &  & \\
 & \frac{1}{10^{2}} &  & \\
 &  & \ddots  & \\
 &  &  & \frac{1}{10^{n}}
\end{bmatrix}\begin{bmatrix}
x_{1}\\
x_{2}\\
\vdots \\
x_{n}
\end{bmatrix} \ -\ \begin{bmatrix}
\frac{2}{10} & \frac{4}{10^{2}} & ... & \frac{2n}{10^{n}}
\end{bmatrix}\begin{bmatrix}
x_{1}\\
x_{2}\\
\vdots \\
x_{n}
\end{bmatrix} \ +\left(\frac{1}{10} +\frac{2^{2}}{10^{2}} +\dotsc +\frac{n^{2}}{10^{n}} \ \right)

\\ =x^TA \ x - b^T x + C
\end{array}



In [89]:
# defining a function to compute step length using the closed form expression 

def compute_steplength_exact(N, g, A):
  assert type(g) is np.ndarray and len(g) == N
  assert type(A) is np.ndarray and A.shape[0] == N and  A.shape[1] == N

  step_length = (np.dot(g.T, g)) / (np.matmul(np.matmul(g.T,2*A ),g))
  return step_length



#defining a function to compute step_length from backtracking

def compute_steplength_backtracking(N, x, gradf, alpha_start, rho, gamma):
  assert type(x) is np.ndarray and len(gradf) == N
  assert type(gradf) is np.ndarray and len(gradf) == N
  
  alpha = alpha_start
  p = -gradf

  while (evalf(N,x + alpha*p) > (evalf(N,x) + gamma * alpha * np.dot(gradf.T, p)) ):
    alpha = alpha*rho
  
  return alpha






EXACT_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH = 2
CONSTANT_STEP_LENGTH = 3


def find_minimizer(N,start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == N #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  # construct a suitable A matrix for the quadratic function 

  A =np.zeros((N,N))

  for i in range(N):
    A[i][i] = 10**(-i-1)
    
  x = start_x
  g_x = evalg(N,x)

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    #print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)

  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol):
     #continue as long as the norm of gradient is not close to zero upto a tolerance tol
  
    if line_search_type == EXACT_LINE_SEARCH:
      step_length = compute_steplength_exact(N,g_x, A) #call the new function you wrote to compute the steplength
      
      #raise ValueError('EXACT LINE SEARCH NOT YET IMPLEMENTED')

    elif line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking(N,x,g_x, alpha_start,rho, gamma) #call the new function you wrote to compute the steplength
      
      #raise ValueError('BACKTRACKING LINE SEARCH NOT YET IMPLEMENTED')

    #elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      #step_length = 0.1

    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(N,x) #compute gradient at new point

    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x,k 


    
  



# ***Part 2 :*** *Evaluating the minimzer and the minimum value using ELS and BLS*





In [90]:
x_start = np.array([5,5,5])
my_tol = 10e-9
alpha = 1
rho = 0.5
gamma = 0.5

x_els, k_els = find_minimizer(3,x_start,my_tol,EXACT_LINE_SEARCH)
x_bls, k_bls = find_minimizer(3,x_start,my_tol,BACKTRACKING_LINE_SEARCH,alpha,rho,gamma)

print('Using Exact Line Search Method - ')
print('Minimizer : ', x_els)
print('The value at the minimizer :', evalf(3,x_els))

print('\n')

print('Using Backtracking Line Search Method - ')
print('Minimizer : ', x_bls)
print('The value at the minimizer :', evalf(3,x_bls))


Using Exact Line Search Method - 
Minimizer :  [0.99999998 2.         3.00000449]
The value at the minimizer : 2.015852775005177e-14


Using Backtracking Line Search Method - 
Minimizer :  [1.         2.         3.00000499]
The value at the minimizer : 2.4915872700821508e-14


# ***Part 3 :*** *Comparing ELS to BLS using the following parameters*

$x^0 = (0.01, 0.1 , 1)$

$\tau = 10^{-9}$

$\alpha ^{0}  =1$

$\rho  = 0.5$

$\gamma  = 0.5$





In [92]:
x_start = np.array([0.01, 0.1, 1])
my_tol = 10e-9
alpha = 1
rho = 0.5
gamma = 0.5

x_els, k_els = find_minimizer(3,x_start,my_tol,EXACT_LINE_SEARCH)
x_bls, k_bls = find_minimizer(3,x_start,my_tol,BACKTRACKING_LINE_SEARCH,alpha,rho,gamma)

print('Using Exact Line Search Method - ')
print('No. of iterations performed : ', k_els)


print('\n')

print('Using Backtracking Line Search Method - ')
print('No. of iterations performed : ', k_bls)



Using Exact Line Search Method - 
No. of iterations performed :  209


Using Backtracking Line Search Method - 
No. of iterations performed :  6444


***Remarks :*** *We observe that as the number of iteration in ELS is very less. That is the answer converge very fast in this alogithm than baktracking which takes a lot of iterations. Given the form of the function, ELS finds the optimum steplength where backtracking iterate again and again to find the solution*

# ***Part 4 :*** *Comparing ELS to BLS using the following parameters*

$x^0 = (0.001,0.01, 0.1 , 1)$

$\tau = 10^{-9}$

$\alpha ^{0}  =1$

$\rho  = 0.5$

$\gamma  = 0.5$





In [93]:
x_start = np.array([0.001,0.01, 0.1, 1])
my_tol = 10e-9
alpha = 1
rho = 0.5
gamma = 0.5

x_els, k_els = find_minimizer(4,x_start,my_tol,EXACT_LINE_SEARCH)
x_bls, k_bls = find_minimizer(4,x_start,my_tol,BACKTRACKING_LINE_SEARCH,alpha,rho,gamma)

print('Using Exact Line Search Method - ')
print('No. of iterations performed : ', k_els)


print('\n')

print('Using Backtracking Line Search Method - ')
print('No. of iterations performed : ', k_bls)



Using Exact Line Search Method - 
No. of iterations performed :  1861


Using Backtracking Line Search Method - 
No. of iterations performed :  55005


***Remarks :*** *Yes, the similar observation holds when N=4. And we observe that again els algorith outperforms BLS in the number of iterationn done to find the optimal answer.*

# ***Part 5 :*** *For when N>4*


***Remarks :*** *Since we have already mentioned it above. The reason ELS ourtperforms better than BLS is that the close form solution for the optimal value of the step length exist in the ELS. And that value exist because we have chosen a quadrratic function where such a solution is possible by solving an optimizationn problem for eta.* ***Even for N>4, if we choose this function, ELS will still outperform BLS and as the dimension of the problem increase we hope to see even  bigger difference in the number of iterations performed.***

